In [1]:
# Python 3 notebook

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
import imageio
import ffmpy
import numpy as np
import pandas as pd
from numpy import fft as fft
from scipy.fftpack import fft, ifft, rfft, irfft, fftfreq, fftshift
from scipy import fftpack
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline
import scipy.io.wavfile
from scipy import signal as signal
from scipy.signal import butter, lfilter
import pylab
from pylab import *
import IPython.display
import os
from sklearn.decomposition import PCA
import csv
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# # Nussl
# import nussl

# # Pyfasst
# import setuptools
# from distutils.core import setup
# from distutils.extension import Extension
# import pyfasst.audioModel as am

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display
from __future__ import print_function

Using TensorFlow backend.


## Target for 10.28.17

1. Cull through training labels and pull out fft segments for each relevant time stamp and period. Put this into a dataframe.

2. Take data from the dataframe and build some way of predicting, when going through an entire game, whether a sequence is a goal or etc.

3. Apply the predictive model to an actual game.

4. [STRETCH] Attempt to take the model's prediction and output a series of clips, based on the model's predictions.

In [2]:
# This will generate a list of lists, that includes filename, timestamp, and fft information
training_games = []

# Import files referenced specifically in the training set

for files in os.listdir('/Users/brianschneider/Desktop/capstone/gameSummarizer/audiofiles/'):
    if files.endswith(".wav"): 
        fileName = '/Users/brianschneider/Desktop/capstone/gameSummarizer/audiofiles/'+files
        samprate, wavdata = scipy.io.wavfile.read(fileName)
        duration = int(len(wavdata)/samprate) # len(wavdata)
        for i in range(0, duration):
            seconds_to_extract = 1
            N = seconds_to_extract * samprate

            # Extracts fourier transform data from the sample, returning the spectrum analysis
            subdata = wavdata[i*samprate:(i+1)*samprate]
            F = scipy.fftpack.fft(subdata)
            F = np.abs(F)
            # Removes imaginary numbers
#             F = [np.real(c) for c in F]
            game = []
            game.append(files)
            game.append(i)
            game.append(F)
            training_games.append(game)

In [3]:
# import csv
 
# with open('/Users/brianschneider/Desktop/capstone/gameSummarizer/eventDataforFFT - Sheet1.csv') as csvDataFile:
#     training_steps = csv.reader(csvDataFile)
#     for row in training_steps:
#         print(row)

training_steps = pd.read_csv('/Users/brianschneider/Desktop/capstone/gameSummarizer/eventDataforFFT.csv')
# training_steps

In [4]:
final_training_data = []

# iterate through training games by second
for t in training_games:
    # iteration through training steps by second
    for i in range(0, len(training_steps)):
        # if training_game id = training_steps id
        if t[0] == training_steps['fileName'][i]:
            # pull secondsOffset from training_steps
            if t[1] == training_steps['secondOffset'][i]:
                # i want to generate the t's that occ
                    t.append(training_steps['goal'][i])
                    final_training_data.append(t)

In [18]:
df = pd.DataFrame(final_training_data, columns=['fileName', 'offset', 'fftData', 'goal'])
df

,fileName,offset,fftData,goal
0,AUG-BVB 1st half ENG.wav,180,"[[1177.0, 591.0], [1212.0, 586.0], [1158.0, 54...",0
1,AUG-BVB 1st half ENG.wav,181,"[[1479.0, 265.0], [1035.0, 215.0], [457.0, 183...",0
2,AUG-BVB 1st half ENG.wav,182,"[[2262.0, 56.0], [2226.0, 48.0], [2171.0, 29.0...",0
3,AUG-BVB 1st half ENG.wav,183,"[[2399.0, 171.0], [2214.0, 132.0], [2060.0, 88...",0
4,AUG-BVB 1st half ENG.wav,184,"[[1348.0, 262.0], [1369.0, 223.0], [1422.0, 16...",0
5,AUG-BVB 1st half ENG.wav,305,"[[6388.0, 8.0], [6336.0, 18.0], [5839.0, 13.0]...",0
6,AUG-BVB 1st half ENG.wav,306,"[[517.0, 159.0], [296.0, 154.0], [132.0, 150.0...",0
7,AUG-BVB 1st half ENG.wav,307,"[[1219.0, 81.0], [850.0, 96.0], [682.0, 132.0]...",0
8,AUG-BVB 1st half ENG.wav,308,"[[311.0, 263.0], [312.0, 248.0], [678.0, 302.0...",0
9,AUG-BVB 1st half ENG.wav,309,"[[1312.0, 224.0], [1435.0, 123.0], [1600.0, 36...",0


In [7]:
# Goal is to input the event_name and fft data into a predictive model to built a training case.

X = df.iloc[:, 2].values
y = df.iloc[:, 3].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [11]:
print(X_train.shape)
print(X_train.dtype)
print(X_test.shape)
print(X_test.dtype)
print(y_train.shape)
print(y_train.dtype)
print(y_test.shape)
print(y_test.dtype)

(128,)
object
(56,)
object
(128,)
int64
(56,)
int64


In [29]:
model = Sequential()
model.add(Dense(12, input_dim=1,
                    kernel_initializer='uniform',
                    activation='relu'))
model.add(Dense(8,kernel_initializer='uniform',activation='relu'))
model.add(Dense(1,kernel_initializer='uniform',activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, batch_size = 25, epochs = 100)

In [32]:
# classifier = Sequential()
# # Adding the input layer and the first hidden layer
# classifier.add(Dense(activation="relu", kernel_initializer="uniform", units=6, input_dim=1))
# # Adding the second hidden layer
# classifier.add(Dense(activation="relu", units=6, kernel_initializer="uniform"))
# # Adding the output layer
# classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
# # Compiling Neural Network
# classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# # Fitting our model 
# classifier.fit(X_train, y_train, batch_size = 20, epochs = 100)
# # Predicting the Test set results
# y_pred = classifier.predict(X_test)
# y_pred = (y_pred > 0.5)
# # Creating the Confusion Matrix
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(y_test, y_pred)

In [22]:
# from keras.layers import Activation,Conv2D,MaxPooling2D,Flatten 
# model2 = Sequential()
# model2.add(Conv2D(32,(3,3),padding='same',input_shape=X_train.shape[1:])) 
# model2.add(Activation('relu'))
# model2.add(Conv2D(32,(3,3)))
# model2.add(Activation('relu'))
# model2.add(MaxPooling2D(pool_size=(2,2)))
# model2.add(Dropout(0.25))
# model2.add(Conv2D(64,(3,3), padding='same'))
# model2.add(Activation('relu'))
# model2.add(Conv2D(64,(3, 3)))
# model2.add(Activation('relu'))
# model2.add(MaxPooling2D(pool_size=(2,2)))
# model2.add(Dropout(0.25))
# model2.add(Flatten())
# model2.add(Dense(512))
# model2.add(Activation('relu'))
# model2.add(Dropout(0.5))
# model2.add(Dense(num_classes))
# model2.add(Activation('softmax'))

## APPENDIX

## This will create 2 channels from a wav input.

In [ ]:
# from scipy.io.wavfile import read
# samprate, wavdata = read('/Users/brianschneider/Desktop/capstone/gameSummarizer/output_audio.wav')
# channel1=wavdata[:,0] #left
# channel2=wavdata[:,1] #right

# print("Sample rate:{0}, data size:{1}, duration:{2} seconds".format(samprate,channel1.shape,len(channel1)/samprate))

## My first goal is to take a fourier transform and break it into seconds and record an array of samples from each second. These will be x-labels.

## Comment out to write a CSV that generates all relevant x-label information. [Make sure you get the directory right!]

In [ ]:
# # data_size = channel1.shape
# # for i in range(0,duration):
# #     test.append(i)
    
# # Loads wav file as array.
# # fileName = '/Users/brianschneider/Desktop/capstone/gameSummarizer/output_audio.wav'
# # sample_rate, data = read(fileName)
 
# # Creating a loop to extract the samples from each second of each game in my audio directory.

# test = []
# # arrays = []

# # dataCSV = open('ProgramCheck1.csv', 'w') 
# # writer = csv.writer(dataCSV, dialect='excel')
# # writer.writerow(['File', 'Second', 'Samples']) 

# # df = pd.DataFrame(columns=('file', 'second', 'array'))

# for files in os.listdir('/Users/brianschneider/Desktop/capstone/gameSummarizer/audiofiles/'):
#     if files.endswith(".wav"): 
#         fileName = '/Users/brianschneider/Desktop/capstone/gameSummarizer/audiofiles/'+files
#         samprate, wavdata = scipy.io.wavfile.read(fileName)
# #         channel1=wavdata[:,0] #left
#         duration = len(wavdata) #int(len(channel1)/samprate)
# #         channel2=wavdata[:,1] #right
#         for i in range(0, duration):
#             seconds_to_extract = 1
#             N = seconds_to_extract * samprate

#             # Knowing N and sample rate, fftfreq gets the frequency
#             # Associated to each FFT unit.
#             f = scipy.fftpack.fftfreq(N, 1.0/samprate)

#             # Extracts fourier transform data from the sample
#             # returning the spectrum analysis
#             subdata = wavdata[i*samprate:(i+1)*samprate]
#             F = scipy.fftpack.fft(subdata)
#             F = np.abs(F)
# #             F = [np.real(c) for c in F]
#             game = []
#             game.append(files)
#             game.append(i)
#             game.append(F)
#             test.append(game)
# #             arrays.append(F)
# #             writer.writerow([files, i, np.array(F)])
            
# #             df = df.append({"file": files, "second":  i, "array": F}, ignore_index=True)

In [ ]:
# # # print(test[0:10])
# # # len(test)

# with open('ProgramCheck1.csv', 'rb') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         print(row)

# f = scipy.fftpack.fftfreq(N, 1.0/samprate)
# f

In [ ]:
# # df3 = pd.read_csv('ProgramCheck1.csv')
# # df3.head()

# plt.plot(test[2][2])

In [ ]:
# df.head()
# # df['Freq'].dtype

In [ ]:
# plt.plot(test[0][2], color='#ff7f00')
# plt.xlabel('k')
# plt.ylabel('Amplitude')
# plt.show()

In [ ]:
# plt.plot(test[1][2], color='#ff7f00')
# plt.xlabel('k')
# plt.ylabel('Amplitude')
# plt.show()

In [ ]:
# # df["Second"] = pd.to_numeric(df["Second"])
# # df["Freq"] = pd.to_numeric(df["Freq"])
# # df["Samples"] = pd.to_numeric(df["Samples"])

# # print(test[0][2].dtype)

# df1 = pd.DataFrame(test, columns=('file', 'second', 'fftSamples'))
# # df1.to_csv("submission2.csv", index=False)

# # plt.plot(df1['fftSamples'][0])

In [ ]:
# df1.head()
# # df2 = pd.read_csv('submission2.csv')

In [ ]:
# f

In [ ]:
# #charts fftfreq by fft

# yf = test[2][2]
# xf = f
# xf = fftshift(xf)
# yplot = fftshift(yf)
# plt.plot(xf, 1.0/44100 * np.abs(yplot))
# plt.grid()
# plt.show()

In [ ]:
# df1['fftSamples'][2][40000:]

In [ ]:
# df1['fftSamples'][2].shape

In [ ]:
# # # https://stackoverflow.com/questions/31794782/numpys-complex128-conversion
# # # [c.real for c in test]
# plt.plot(df1['fftSamples'][2])

In [ ]:
# plt.plot(df1['fftSamples'][7])
# plt.semilogy(df1['fftSamples'][2])

In [ ]:
# shape(arrays)

In [ ]:
# pca = PCA(n_components=3)
# pca.fit(test[1][2])
# arrays = pca.transform(arrays)

## My second goal is to create a y-label column based on goals at specific time stamps in a game. I will add these manually w/ a binary flag.
## My third goal is to train a data set based on x-labels and y-labels.
## My fourth goal is to try and apply the model to a test game and see how the predictions fare.

## Do I want to try PCA before moving too far ahead?

## http://gonzalopla.com/python-for-digital-signal-processing/

This example does what I want to do.

## https://pypi.python.org/pypi/QSpectrumAnalyzer

Also worth checking out if it has an output function.

## http://pythonforengineers.com/audio-and-digital-signal-processingdsp-in-python/

Might be worth a look.

In [ ]:
# print(wave_file_list[1].shape)

In [ ]:
# Y = channel1.reshape(-1, 1)
# pca = PCA(n_components=0)
# pca.fit(Y)
# tyr = PCA(copy=True, iterated_power='auto', n_components=0, random_state=None,
#   svd_solver='full', tol=0.0, whiten=False)
# print(tyr)
# # print(pca.explained_variance_ratio_)  
# # # [ 0.99244...  0.00755...]
# # print(pca.singular_values_)  
# # # [ 6.30061...  0.54980...]

In [ ]:
# pca = PCA(n_components=2)
# pca.fit(fourier)   
# PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
#   svd_solver='auto', tol=0.0, whiten=False)
# print(pca.explained_variance_ratio_)  
# print(pca.singular_values_) 

## APPENDIX

In [ ]:
# #http://myinspirationinformation.com/uncategorized/audio-signals-in-python/
# from scipy.io.wavfile import read
# wave_file_list = []
# for filename in os.listdir('/Users/brianschneider/Desktop/capstone/gameSummarizer/audiofiles'):
#     if filename.endswith(".wav"): 
#         samprate, wavdata = read(filename)
#         channel1=wavdata[:,0] #left
#         channel2=wavdata[:,1] #right
#         # time = np.arange(0, float(wavdata.shape[0]), 1) / samprate
#         fourier=np.fft.fft(channel1)
#         n = channel1.size
#         timestep = 1.0
#         freq = np.fft.fftfreq(n, d=timestep)
#         wave_file_list.append(freq)
#         len(freq)
#         continue
#     else:
#         continue

In [ ]:
# import scipy.io
 
# # Package that implements the fast
# # fourier transform functions.
# from scipy import fftpack
# import numpy as np
 
# # Loads wav file as array.
# fileName = '/Users/brianschneider/Desktop/capstone/gameSummarizer/output_audio.wav'
# sample_rate, data = read(fileName)
 
# # Extracting 10 seconds. N is the numbers of samples to
# # extract or elements from the array.
# seconds_to_extract = 10
# N = seconds_to_extract * sample_rate
 
# # Knowing N and sample rate, fftfreq gets the frequency
# # Associated to each FFT unit.
# f = fftpack.fftfreq(N, 1.0/sample_rate)
 
# # Extracts fourier transform data from the sample
# # returning the spectrum analysis
# subdata = data[:N]
# F = fftpack.fft(subdata)

# fig, ax = plt.subplots()

# ax.stem(f, np.abs(F))
# ax.set_xlabel('Frequency in Hertz [Hz]')
# ax.set_ylabel('Frequency Domain (Spectrum) Magnitude')
# ax.set_xlim(-sample_rate / 2, sample_rate / 2)
# ax.set_ylim(-5, 110)

In [ ]:
# f_s = 100
# signal = channel1
# fourier = np.fft.fft(signal)
# n = signal.size
# timestep = 1.0
# freqs = np.fft.fftfreq(n, d=timestep)
# # freq

# fig, ax = plt.subplots()

# ax.stem(freqs, np.abs(fourier))
# ax.set_xlabel('Frequency in Hertz [Hz]')
# ax.set_ylabel('Frequency Domain (Spectrum) Magnitude')
# ax.set_xlim(-f_s / 2, f_s / 2)
# ax.set_ylim(-5, 110)
